In [1]:
from glob import glob
import xml.etree.ElementTree as ET

import cv2
import numpy as np
from keras.applications import ResNet50
from keras.layers import Dense, Flatten, GlobalAveragePooling2D
from keras.models import Sequential
from sklearn.model_selection import train_test_split


In [2]:
images = []
annotations = []

for img, annotation in zip(glob('../../dataset/images/*.png'), glob('../../dataset/annotations/*.xml')):
    image = cv2.imread(img)

    xml = ET.parse(annotation)
    info = xml.getroot().find('object').find('bndbox')
    xmin = int(info.find('xmin').text)
    ymin = int(info.find('ymin').text)
    xmax = int(info.find('xmax').text)
    ymax = int(info.find('ymax').text)

    shape = image.shape
    xmin = xmin / shape[1]
    xmax = xmax / shape[1]
    ymin = ymin / shape[0]
    ymax = ymax / shape[0]

    image = cv2.resize(image, (224, 224))

    images.append(image)
    annotations.append((xmin, ymin, xmax, ymax))

images = np.array(images) / 255
annotations = np.array(annotations)


In [3]:
x_train, x_test, y_train, y_test = train_test_split(images, annotations, test_size=0.2, random_state=42)
x_test, x_val, y_test, y_val = train_test_split(x_test, y_test, test_size=0.5, random_state=42)


In [4]:
model = Sequential()
model.add(ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3)))
model.add(GlobalAveragePooling2D())
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dense(4))

model.layers[0].trainable = False

model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 flatten (Flatten)           (None, 2048)              0         
                                                                 
 dense (Dense)               (None, 1024)              2098176   
                                                                 
 dense_1 (Dense)             (None, 4)                 4100      
                                                                 
Total params: 25,689,988
Trainable params: 2,102,276
Non-trainable params: 23,587,712
____________________________________

In [5]:
model.compile(optimizer='adam', loss='mse')
history = model.fit(np.array(x_train), np.array(y_train), validation_data=(np.array(x_val), np.array(y_val)), epochs=500)


Epoch 1/500


KeyboardInterrupt: 

In [ ]:
model.save('./model_500.h5')
